Laser Sources

In [1]:
import asyncio
import neaspec
import configuration as config

context = await neaspec.connect(config.db, config.vm, config.dll)

> Redirecting unknown loading assembly System.Runtime.CompilerServices.Unsafe, Version=4.0.4.1, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a
> Configuration in local file C:\Users\Patrick\AppData\Local\Python_Software_Foundatio\DefaultDomain_Path_mndturxegrmy5qhqcgfnwau3k5cajhfi\3.7.7\user.config
> Loaded from database group: Nea.Client.Hardware.Microscope.Properties.Settings
> Loaded from database group: Nea.Client.Hardware.Camera.Properties.Settings
> No camera found. Discovery result: Success
> Waiting for initialization...
> Trying to connect to 127.0.0.1
> Trying to connect to 127.0.0.1
> Trying to connect to 127.0.0.1
> Connected to 127.0.0.1
> Connected to 127.0.0.1
> Connected to 127.0.0.1
> Receive thread (Nea.Client.Hardware.MotorRpc) started.
> Receive thread (Nea.Client.Hardware.Rpc) started.
> Receive thread (Nea.Client.Hardware.PreviewRpc) started.
> Client 1.10.7123 is incompatible with Server 1.10.7131.
> Client 1.10.7123 is incompatible with Server 1.10.7131.
> Cli

The property `AvailableLaserSources` provides all laser sources that are available in the current sphere.

In [12]:
# shows a dictionary, with laser soruce id as key 
print(*context.Logic.AvailableLaserSources)

[Mule, Nea.Client.Hardware.LaserSource.Mule] [MuleSCIR, Nea.Client.Hardware.LaserSource.Mule] [FemtoFiberArm1, Nea.Client.Hardware.LaserSource.FemtoFiber] [MuleIR, Nea.Client.Hardware.LaserSource.Mule]


In [13]:
import Nea.Client.SharedDefinitions as nea
mule = context.Logic.AvailableLaserSources[nea.LaserSourceId.Mule]
print(mule.Name)
print(mule.Wavelength)

success, femtofiber = context.Logic.AvailableLaserSources.TryGetValue(nea.LaserSourceId.FemtoFiberArm1, None)
if success:
    print(femtofiber.Name)
else:
    print("No femtofiber")

4.5-15 µm nano‑FTIR Laser
None
1.55 µm nano‑FTIR laser


In order to select a laser source:

In [14]:
context.Logic.ChangeLaserSource.Execute(mule)

> Approving changing laser source is unhandled and will be refused!


For safety some laser sources or laser source configurations require the user to confirm the activation. 
In the neaSCAN application a popup windows would appear. In those cases where the script runs independently the selection fails with an error message. You can register callbacks in order to implement your own user approval.

In [21]:
# This function will be called whenever changing the laser source
# needs user approval (e.g. safety glasses, filters, etc...) 
def onApprovingChangingLaserSource(args) -> sdk.TaskOf[bool]:
    print(f'Do you really want to change the laser source?')
    print(f'{args.OldValue} -> {args.NewValue}')

    task = sdk.TaskOf[bool].FromResult(False) # or True
    # Asynchronously prompt user YES/NO
    
    print(f'Approved: {task.Result}');
    return task


context.Logic.ApprovingChangingLaserSource = nea.ApprovingChangingLaserSource(onApprovingChangingLaserSource)

context.Logic.ChangeLaserSource.Execute(mule)
print(f'Laser source after change: {context.Logic.LaserSource}')

Do you really want to change the laser source?
None -> Nea.Client.Hardware.LaserSource.Mule
Approved: False
Laser source after change: None
